## **Proyecto - Statistical Learning I**

### Desarrollo del Proyecto

#### Paquetes a utilizar

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from sklearn.feature_selection import SelectKBest
from sklearn.model_selection import train_test_split
from sklearn import tree
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.metrics import mean_squared_error
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import f1_score
from sklearn.preprocessing import PolynomialFeatures
from joblib import dump
import os
import math
from scipy.stats import norm
from datetime import datetime

In [2]:
if tf.__version__.startswith("2."):
    import tensorflow.compat.v1 as tf
    tf.compat.v1.disable_v2_behavior()
    tf.compat.v1.disable_eager_execution()

Instructions for updating:
non-resource variables are not supported in the long term


#### Carga de datos

In [135]:
data_titanic = pd.read_csv('data_titanic_proyecto.csv')
data_titanic

,PassengerId,Name,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked,passenger_class,passenger_sex,passenger_survived
0,1,"Braund, Mr. Owen Harris",22.0,1,0,A/5 21171,7.2500,NaN,S,Lower,M,N
1,2,"Cumings, Mrs. John Bradley (Florence Briggs Th...",38.0,1,0,PC 17599,71.2833,C85,C,Upper,F,Y
2,3,"Heikkinen, Miss. Laina",26.0,0,0,STON/O2. 3101282,7.9250,NaN,S,Lower,F,Y
3,4,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",35.0,1,0,113803,53.1000,C123,S,Upper,F,Y
4,5,"Allen, Mr. William Henry",35.0,0,0,373450,8.0500,NaN,S,Lower,M,N
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,"Montvila, Rev. Juozas",27.0,0,0,211536,13.0000,NaN,S,Middle,M,N
887,888,"Graham, Miss. Margaret Edith",19.0,0,0,112053,30.0000,B42,S,Upper,F,Y
888,889,"Johnston, Miss. Catherine Helen ""Carrie""",NaN,1,2,W./C. 6607,23.4500,NaN,S,Lower,F,N
889,890,"Behr, Mr. Karl Howell",26.0,0,0,111369,30.0000,C148,C,Upper,M,Y


#### Limpieza de datos

Convertir valores NaN a cero

In [136]:
data_titanic = data_titanic.fillna(0)

#### Selección de Variables

Posibles variables predictoras
- Age
- Fare
- passenger_class
- passenger_sex

Variable a predecir

- passenger_survived

#### Conversión de variables para determinar nivel de correlación

A continuación se convierten aquellas variables categóricas a un factor númerico, esta conversión se hace únicamente para determinar el nivel de correlación de las variables independientes con la variable dependiente, pero no es una transformación de encoded para el proceso de entrenamiento.

*La variable Age no requiere conversión ya que por defecto su valor es númerico*

In [137]:
data_titanic['passenger_survived_codes'] = data_titanic['passenger_survived'].astype('category').cat.codes
data_titanic['passenger_sex_codes'] = data_titanic['passenger_sex'].astype('category').cat.codes
data_titanic['passenger_class_codes'] = data_titanic['passenger_class'].astype('category').cat.codes

#### Correlación de variables

In [138]:
data_titanic[data_titanic.columns[1:]].corr()['passenger_survived_codes'][:]

Age                         0.010539
SibSp                      -0.035322
Parch                       0.081629
Fare                        0.257307
passenger_survived_codes    1.000000
passenger_sex_codes        -0.543351
passenger_class_codes       0.338481
Name: passenger_survived_codes, dtype: float64

#### Depuración de features

Se eliminan aquellas características que son identificadores, nombres o etiquetas.

In [139]:
X=data_titanic.drop(['passenger_survived_codes','PassengerId', 'Name', 'Ticket', 'Cabin', 'Embarked', 'passenger_class', 'passenger_sex', 'passenger_survived'], axis=1)
Y=data_titanic['passenger_survived_codes']

#### Selección de mejores features

Utilizando la libreria "SelectKBest" se determinan las 3 mejores características y sobre esas se trabajan. 

In [140]:
best=SelectKBest(k=3)
best.fit_transform(X, Y)
selected = best.get_support(indices=True)
print(X.columns[selected])

Index(['Fare', 'passenger_sex_codes', 'passenger_class_codes'], dtype='object')


Se utilizarán las variables:

- Fare
- passenger_sex
- passenger_class

Ya que poseen un alto nivel de correlación y según la libreria "SelectKBest" las sugiere como mejores features.

In [141]:
used_features = X.columns[selected]

De la posibles features "X", se eliminan aquellas que no formaran parte del proceso de predicción.

In [142]:
X = X.drop(['Age', 'SibSp', 'Parch'], axis=1)

#### División de datos para entreno, validación y prueba

In [145]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)
x_train, x_validate, y_train, y_validate = train_test_split(x_train, y_train, test_size=0.3)

### Definición de funciones generales

#### Función para crear el archivo CSV que servirá de bitácora

In [12]:
bitacora = {'string_config': [],
            'accuracy': [],
            'error': [],
            'precision': [],
            'recall': [],
            'f1': []}

df = pd.DataFrame(bitacora, columns= ['string_config', 'accuracy', 'error', 'precision', 'recall', 'f1'])
df.to_csv('bitacora.csv', index = False, header=True)

#### Función para reescribir el CSV de bitácora

In [13]:
def set_record(string_config, metrics):
    bitacora = {'string_config': [string_config],
                'accuracy': [metrics[0]],
                'error': [metrics[1]],
                'precision': [metrics[2]],
                'recall': [metrics[3]],
                'f1': [metrics[4]]}
    bitacora = pd.DataFrame(bitacora, columns= ['string_config', 'accuracy', 'error', 'precision', 'recall', 'f1'])
    bitacora.to_csv('bitacora.csv', mode='a', index = False, header=False)

#### Función que devuelve el string de métricas

In [14]:
def get_str_metrics(str_model, mtr):
    return "Model: "+str_model+" | Accuracy: "+get_percent(mtr[0])+" | Error: "+get_percent(mtr[1])+" | Precisión: "+get_percent(mtr[2])+" | Recall: "+get_percent(mtr[3])+" | F1: "+get_percent(mtr[4])

#### Función que da formato de porcentaje a una cantidad

In [16]:
def get_percent(number):
    return str(round((number*100),2))+"%"

#### Función para generación de métricas

In [17]:
def get_metrics(y_true, y_predict):
    accuracy = accuracy_score(y_true, y_predict)
    error = mean_squared_error(y_true, y_predict)
    precision = precision_score(y_true, y_predict, average='weighted')
    recall = recall_score(y_true, y_predict, average='weighted')
    f1 = f1_score(y_true, y_predict, average="weighted")
    
    return accuracy, error, precision, recall, f1

#### Función para guardar el modelo y registrar bitácora

In [18]:
def generate_model_x_record(str_config, str_model, model, metrics, isSkLearn):
    print(get_str_metrics(str_model, metrics))
    set_record(str_config, metrics)
    path = "modelos/"+str_config+"_"+datetime.now().strftime("%Y%m%d_%H%M%S")
    if(isSkLearn):
        dump(model, path+".joblib")
    else:
        if (str_model == "Naive Bayes"):
            os.mkdir(path)
            model["mean"].to_csv(path+"/mean.csv", index = False, header=True)
            model["stdev"].to_csv(path+"/stdev.csv", index = False, header=True)        
            pd.DataFrame(data=model["probabilities"][0:2], columns=["probabilities"]).to_csv(path+"/probabilities.csv", index = False, header=True)
            pd.DataFrame(data=model["class"][0:2], columns=["class"]).to_csv(path+"/class.csv", index = False, header=True)
        else:
            pd.DataFrame(data=model[0], columns=["params"]).to_csv(path+".csv", index = False, header=True)

#### Función para determinar la moda de un conjunto de datos

In [156]:
def moda(data):
    repetitions = 0
    moda = -1
    for i in data:
        n = data.count(i)
        if n > repetitions:
            repetitions = n
    for i in data:
        n = data.count(i)
        if n == repetitions and moda == -1:
            moda = i     
    return moda

#### Función para obtener predición final en base a la moda de las predicciones individuales

In [157]:
def get_final_prediction(prediction_joined):
    moda_prediction = []
    for predict in prediction_joined:
        moda_predict = moda(list(predict))
        moda_prediction.append(moda_predict)
    return np.array(moda_prediction)

### Definción de Modelos

#### Árbol de decisión

In [19]:
def model_decision_tree(x_train, y_train, x_validate, y_validate):
    
    tree_model = tree.DecisionTreeClassifier()
    tree_model = tree_model.fit(x_train, y_train)
    y_predict = tree_model.predict(x_validate)
    
    return y_predict, tree_model, get_metrics(y_validate, y_predict)

#### SVM

In [20]:
def model_svm(x_train, y_train, x_validate, y_validate):

    svm_model = svm.SVC()
    svm_model = svm_model.fit(x_train, y_train)
    y_predict = svm_model.predict(x_validate)
    
    return y_predict, svm_model, get_metrics(y_validate, y_predict)

#### Naive Bayes

In [164]:
def predict_naive_bayes(model, x_validate):
    y_predict = []
    for i in range(x_validate.shape[0]):
        probability={}
        for y_class in model["class"]:
            probability[y_class] = model["probabilities"].iloc[y_class]
            for index, _ in enumerate(x_validate.iloc[i]):
                probability[y_class] *= norm.pdf(x_validate.iloc[i], model["mean"].iloc[y_class, index], model["stdev"].iloc[y_class, index])
        y_predict.append(get_argmax(probability))
    return y_predict

In [22]:
def get_argmax(probability):
    max_value = 0
    argmax = -1
    for (key, value) in probability.items():
        if (key == 0):
            max_value = max(value)
            argmax = key
        else:
            tmp = max(value)
            if(max_value < tmp):
                max_value = tmp
                argmax = key
    return argmax

In [23]:
def model_naive_bayes(x_train, y_train, x_validate, y_validate):
    
    mean = x_train.groupby(y_train).apply(np.mean)
    stdev = x_train.groupby(y_train).apply(np.std)
    probabilities = x_train.groupby(y_train).apply(lambda x: len(x) / x_train.shape[0])
    y_class = np.unique(y_train)
    bayes_model = {"mean":mean, "stdev":stdev, "probabilities":probabilities, "class":y_class}
    y_predict = predict_naive_bayes(bayes_model, x_validate)
    
    return y_predict, bayes_model, get_metrics(y_validate, y_predict)

#### Regresión Logística

In [24]:
def sigmoid(x):
    return 1 / (1 + math.exp(-x))

In [167]:
def predict_reg_logistic(model, x_validate):
    y_predict = []
    for feature in x_validate.values:
        value = 0
        for i in range(len(feature)):
            value += feature[i] * model[i][0]
        value_sigmoid = sigmoid(value)
        if value_sigmoid >= 0.5:
            y_predict.append(1)
        else:
            y_predict.append(0)
    return y_predict

#### Definición del Grafo

In [70]:
tf.reset_default_graph()

weight = tf.Variable(tf.truncated_normal([2, 1]), name = "weight", dtype = tf.float32)
bias = tf.Variable(tf.zeros([]), name = "bias", dtype = tf.float32)

learning_rate = tf.placeholder(shape = [], name = "learning_rate", dtype = tf.float32)
factor_reg = tf.placeholder(tf.float32)
tensor_x = tf.placeholder(shape = [None, 2], name = "tensor_x", dtype = tf.float32)
tensor_y = tf.placeholder(shape = [None, 1], name = "tensor_y", dtype = tf.float32)

with tf.name_scope("logits"):
    logits = tf.matmul(tensor_x, weight) + bias
    
with tf.name_scope("cross_entropy"):
    regularization = tf.nn.l2_loss(weight);
    cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = tensor_y)) + (factor_reg*regularization)
    cross_entropy_summary = tf.summary.scalar(name="cross_entropy",tensor=cross_entropy)

with tf.name_scope("accuracy"):
    accuracy = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits,1), tf.argmax(tensor_y,1)), tf.float32))
    accuracy_summary = tf.summary.scalar(name="accuracy",tensor=accuracy)

with tf.name_scope("gradient"):
    gradient = tf.gradients(cross_entropy, weight)

with tf.name_scope("new_weight"):
    new_weight = tf.assign(weight, weight - learning_rate * gradient[0])

init = tf.global_variables_initializer()

#### Mini Batch Gradient Descent

In [71]:
def model_reg_logistic(x_train, y_train, x_validate, y_validate, epochs, batch_size, total_iter, lr, rg):
    with tf.train.MonitoredSession() as session:
        session.run(init)
        for epoch in range(epochs):
            for i in range(total_iter):
                start_index = i*batch_size
                end_index = start_index+batch_size
                x = np.array(x_train[start_index:end_index])
                y = np.array(y_train[start_index:end_index]).reshape(batch_size,1)

                feed_dict = {tensor_x:x, tensor_y:y, learning_rate:lr, factor_reg:rg}
                _, c, a, w, b= session.run([new_weight, cross_entropy, accuracy, weight, bias],feed_dict=feed_dict)
    reg_logistic_model = [w,b]
    y_predict = predict_reg_logistic(reg_logistic_model, x_validate)
        
    return y_predict, reg_logistic_model, get_metrics(y_validate, y_predict)

### Experimentos

#### Árbol de decisión

In [116]:
prediction, tree_model, metrics = model_decision_tree(x_train[used_features], y_train, x_validate[used_features], y_validate)
generate_model_x_record("decisionTreeLog_varFare_varSex_varClass", "Decision Tree", model, metrics, True)

Model: Decision Tree | Accuracy: 81.78% | Error: 18.22% | Precisión: 81.55% | Recall: 81.78% | F1: 81.57%


#### SVM

In [128]:
prediction, svm_model, metrics = model_svm(x_train[used_features], y_train, x_validate[used_features], y_validate)
generate_model_x_record("svmLog_varFare_varSex_varClass", "SVM", model, metrics, True)

Model: SVM | Accuracy: 80.37% | Error: 19.63% | Precisión: 80.13% | Recall: 80.37% | F1: 80.19%


#### Naive Bayes

In [146]:
prediction, bayes_model, metrics = model_naive_bayes(x_train[used_features], y_train, x_validate[used_features], y_validate)
generate_model_x_record("naiveBayesLog_varFare_varSex_varClass", "Naive Bayes", model, metrics, False)

Model: Naive Bayes | Accuracy: 76.17% | Error: 23.83% | Precisión: 78.91% | Recall: 76.17% | F1: 74.37%


#### Regresión Logística

In [93]:
batch_size = 20
epochs = 5
lr= 0.0001
regularization = 0.01
sample_size = len(x_train)
total_iter = int(sample_size / batch_size)

prediction, reg_logistic_model, metrics = model_reg_logistic(x_train.values, y_train.values, x_validate, y_validate, 
                                                epochs, batch_size, total_iter, lr, regularization)

str_config = "regLogisticLog_lr="+str(lr)+"_reg="+str(regularization)+"_bath_size="+str(batch_size)+"_varFare_varSex_varClass"
generate_model_x_record(str_config, "Regression Logistic", model, metrics, False)

INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
Model: Regression Logistic | Accuracy: 79.91% | Error: 20.09% | Precisión: 79.94% | Recall: 79.91% | F1: 79.61%


### Predicciones con el set de pruebas (todos los modelos)

*Debido a que el modelo generado con SVM y Regresión Logística se hizo en base a dos variables, se debe eliminar del set de datos la característica "Fare"

In [ ]:
x_test_ = x_test.drop(['Fare'], axis=1)

In [177]:
y_predict_tree = tree_model.predict(x_test)
y_predict_svm = svm_model.predict(x_test_)
y_predict_bayes = predict_naive_bayes(bayes_model, x_test)
y_predict_reg_log = predict_reg_logistic(reg_logistic_model.values, x_test_)

#### Combinación de predicciones

In [181]:
y_predict_joined = np.stack((y_predict_tree, y_predict_svm, y_predict_bayes, y_predict_reg_log), axis=-1)

#### Obteniendo predicción en función de la moda

In [182]:
y_moda_predicted = get_final_prediction(y_predict_joined)

#### Tabla de Predicciones

In [184]:
y_prediction_summary = np.stack((y_predict_tree, y_predict_svm, y_predict_bayes, y_predict_reg_log, y_moda_predicted), axis=-1)
df_predictions = pd.DataFrame(y_prediction_summary, columns = ["Decision Tree","SVM","Bayes","Reg. Log.","Moda Predict"])
df_predictions

,Decision Tree,SVM,Bayes,Reg. Log.,Moda Predict
0,0,0,0,0,0
1,0,0,0,0,0
2,0,0,0,0,0
3,1,0,0,0,0
4,0,0,0,0,0
...,...,...,...,...,...
174,0,0,0,0,0
175,0,0,0,0,0
176,1,1,1,1,1
177,0,0,0,0,0


#### Cálculo de Métricas

In [185]:
metrics_tree = get_metrics(y_test, y_predict_tree)
metrics_svm = get_metrics(y_test, y_predict_svm)
metrics_bayes = get_metrics(y_test, y_predict_bayes)
metrics_reg_log = get_metrics(y_test, y_predict_reg_log)
metrics_moda_predicted = get_metrics(y_test, y_moda_predicted)
metrics_joined = np.stack((metrics_tree, metrics_svm, metrics_bayes, metrics_reg_log, metrics_moda_predicted), axis=-1)

#### Tabla de Métricas

In [186]:
df_metrics = pd.DataFrame(metrics_joined, index = ["Accuracy","Error","Precision","Recall","F1"], 
                          columns = ["Decision Tree","SVM","Bayes","Reg. Log.","Moda Predict"])
df_metrics

,Decision Tree,SVM,Bayes,Reg. Log.,Moda Predict
Accuracy,0.877095,0.787709,0.715084,0.787709,0.782123
Error,0.122905,0.212291,0.284916,0.212291,0.217877
Precision,0.879815,0.789249,0.706847,0.789249,0.781436
Recall,0.877095,0.787709,0.715084,0.787709,0.782123
F1,0.873918,0.788378,0.706940,0.788378,0.781754


### Conclusiones

- De acuerdo al número de experimentos que se realizaron se observa que uno de los modelos que ofrece mayor exactitud es Decision Tree empleando para ello tres variables durante el entrenamiento (fare, sex, class).
- Uno de los modelos que más computación parece requerir es Naive Bayes ya que durante el proceso de entreno el tiempo de respuesta es mucho mayo en comparación al resto.
- El modelo de regresión logística requirió bastante experimentación con relación a determinar las variables a utilizar y la definición de los hiper-parametros.
- La combinación de los modelos no resultó ser la mejor de las alternativas, ya que aún así el modelo Decision Tree continua siendo el mejor modelo.


### Anexos

#### Bootstrapping

Este métodlo permite realizar un remuestreo con el objetivo de evitar el sesgo, la idea principal es obtener una muestra con reemplazo de la muestra original N cantidad de veces, donde N viene siendo el tamaño total de la muestra. Al obtener una muestra del mismo tamaño que la original se consigue un estimador y para el caso, deben lograrse varios estimadores.

En este proyecto la manera de implementar Bootstrapping hubiera sido de la siguiente manera:

De la población con un tamaño de 891 se obtiene una muestra, para el caso la muestra aleatoria será de 500 y de esta se buscará obtener varios estimadores. El estimador debe poseer el mismo tamaño de la muestra original y se logra realizando el proceso repetitivo de obtener una muestra aleatoria con reemplazo. Para el caso se supondrá que el remuestreo se hará 100 veces, lo que quiere decir que se obtendrán 100 estimadores y para cada uno habrá un estadístico que servirá para determinar con mayor exactitud la predicción de sobrevivientes.

#### K-Folds Cross Validation

Generalmente al realizar el proceso de muestreo para la creación de modelos se hace diviendo la muestra en entreno y pruebas, el problema de realizarlo de esta manera es que probablemente el set que se utilice para entrenar el modelo pueda poseer un grupo de registros que no sean lo suficientemente útiles para alcanzar un nivel de exactitud alto. En consecuencia surge una técnica de validación cruzada que busca obtener los mejores resultados.

*Validación Cruzada o K-fold Cross Validation* consiste en tomar la muestra original y dividirla en K grupos para realizar el entreno y las pruebas. Este proceso se lleva a cabo de forma iterativa tomando en cuenta que cada K será el número total de iteraciones a ejecutarse, por cada iteración se divide la muestra tomando un grupo de pruebas y el resto para entreno, en la siguiente iteración el grupo que se tomo como prueba ahora será parte del entreno y un grupo distinto ahora será el de prueba, logrando que en toda la muestra todos los grupos en algún momento hayan formando tanto del set de pruebas como de entreno. Este método exige un mayor grado de computación debido a la cantidad de iteraciones y el tamaño de la muestra que se procese.

Para el proyecto la implementación podría ser de la siguiente manera:

- Se toma el set de datos completo que posee un tamaño igual a 891.
- Se define el valor de K en el que se dividirá el set de datos, para el ejemplo se tomará un K=40.
- Se crearan 40 grupos de aproximadamente 15 registros.
- El algoritmo iniciara recorriendo el primer elemento cual será utilizado como pruebas, quiere decir que los 39 grupos restantes servirán para el entrenamiento.
- En la siguiente iteración el grupo que anteriormente sirvió como prueba formará parte del entreno y en su lugar el segundo grupo será ahora de pruebas.
- El proceso se repite hasta completar los 40 grupos en que se dividio el set de datos.

A continuación en la figura se muestra gráficamente como funcionaría el algoritmo.

<img src="https://raw.githubusercontent.com/estuardozapeta/estuardozapeta-Statistical-Learning-I-Proyecto/master/k-folds.jpg">

